In [390]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso,RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

In [2838]:
#Загрузка данных
x_train_load = pd.DataFrame(np.load('x_train.npy'))
y_train_load = pd.DataFrame(np.load('y_train.npy'))
x_test_load = pd.DataFrame(np.load('x_test.npy'))


In [1465]:
#Данные для итогового теста
x_train=x_train_load
y_train=y_train_load
x_test=x_test_load

In [2839]:
#Делим данные для создания модели
x_train, x_test, y_train, y_test = train_test_split(x_train_load, y_train_load, train_size=0.8, random_state=42)

In [2840]:
#Метрика
def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [2841]:
#С помощью Kmean разделил по координатам на области данные(более богатые районы или менее)
xx=pd.DataFrame(x_train.lat)
xx['long']= x_train.long
#model1 = KMeans(n_clusters=6) 
#model1.fit(xx)

all_predictions = model1.predict(xx)
x_train['ll']=all_predictions

xx=pd.DataFrame(x_test.lat)
xx['long']= x_test.long
all_predictions = model1.predict(xx)
x_test['ll']=all_predictions

In [2842]:
del(x_train['date'])
del(x_test['date'])

In [2843]:
# Добавил признак с расстояние до ближайшей достопримечательности, парка и тд. Так как все координаты в приделах Сиэтл
import math

new_featch=[[47.688486, -122.400450],[47.694006, -122.388608],[47.694785, -122.377502],[47.707896, -122.365916],[47.694526, -122.272156],
           [47.689569, -122.279038],[47.676638, -122.261216],[47.681023, -122.328301],[47.667768, -122.351272],[47.667706, -122.398084],
           [47.651418, -122.349172],[47.659994, -122.311291],[47.640433, -122.299133],[47.634983, -122.312434],[47.626867, -122.337600],
           [47.635249, -122.344991],[47.640978, -122.377600],[47.654899, -122.409009],[47.629365, -122.359476],[47.621244, -122.351299],
           [47.607963, -122.339742],[47.603277, -122.330859]]

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    dec_m = 6371 * c
    return dec_m

t=[]
xx=pd.DataFrame(x_train.lat)
xx['long']= x_train.long
for x in xx.values:
    s=[]
    for i in new_featch:
        s.append(haversine(x[1],x[0],i[0],i[1]))
    t.append(min(s))
x_train['new']=t
t=[]
xx=pd.DataFrame(x_test.lat)
xx['long']= x_test.long
for x in xx.values:
    s=[]
    for i in new_featch:
        s.append(haversine(x[1],x[0],i[0],i[1]))
    t.append(min(s))
x_test['new']=t

In [2844]:
#Еще одна кластраризация видов районов и расстояния до ближайших достопримечательностей
x_train['price']= y_train
xx=pd.DataFrame(x_train.new)
xx['zip']= x_train.ll
s=[]
#for i in range(3,12):
#    model = KMeans(n_clusters=5) 
#    model.fit(xx)
#model = KMeans(n_clusters=5) 
#model.fit(xx)
all_predictions = model.predict(xx)
x_train['zl']=all_predictions

xx=pd.DataFrame(x_test.new)
xx['zip']= x_test.ll
all_predictions = model.predict(xx)
x_test['zl']=all_predictions

In [2845]:
del(x_train['sqft_living'])
del(x_test['sqft_living'])

In [2846]:
x_train['sqft_lot']=np.log(x_train['sqft_lot'])
x_test['sqft_lot']=np.log(x_test['sqft_lot'])
x_train['sqft_above']=np.log(x_train['sqft_above'])
x_test['sqft_above']=np.log(x_test['sqft_above'])

In [1]:
#Смотрю как коррелируют признаки
x_train['price']= y_train
x_train.corr().price

In [1761]:
#Строил зависимость признаков
import matplotlib.pyplot as plt
def pairplot(df, target):
    ncol, nrow = 7, df.shape[1] // 7 + (df.shape[1] % 7 > 0)
    plt.figure(figsize=(ncol * 4, nrow * 4))

    for i, feature in enumerate(df.columns):
        plt.subplot(nrow, ncol, i + 1)
        plt.scatter(df[feature], target, s=10, marker='o', alpha=.6)
        plt.xlabel(feature)
        if i % ncol == 0:
            plt.ylabel('target')
pairplot(x_train, y_train)

In [2857]:
#del(x_train['price'])
parameters = {
    'alpha': np.logspace(-6,6,70)#-4,-3,70
}

standard_pipeline1=Pipeline([
    ('poly_feat', PolynomialFeatures(2)),
    ('std_scaler', StandardScaler()),
    ('cv', GridSearchCV(Ridge(), parameters,cv=5,error_score=mape)),    
])


#cv=GridSearchCV(Ridge(), parameters,cv=5)
ll=standard_pipeline1.fit(x_train,np.log(y_train))

test_r=ll.predict(x_test)
print('Ridge(score R2) %.10f\n' % mape(y_test, np.exp(test_r)))

Ridge(score R2) 12.8906056720



In [2821]:
#Сохранение модели
f=pd.DataFrame( np.array(range(1,len(test_r)+1)) )
f.columns = ['Id']
f['Price']=np.exp(test_r)
f.index=f.Id
del(f['Id'])
import csv
f.to_csv('answer.csv')

In [2711]:
#Я устал выбирать признаки, и я создал такой подбор признаков для кластаризации и выбирал по лучшей метрики потом
s=[]
name=[]
for i in x_train.columns:
    print(i)
    for j in x_train.columns:

#for i in range(3,12):
#    model_feat = KMeans(n_clusters=i) 
#    model_feat.fit(xx)
        for k in range(3,5):
            x_train['price']= y_train
            xx=pd.DataFrame(x_train[i])
            xx['zip']= x_train[j]
            model_feat_new_new_new_new = KMeans(n_clusters=k) 
            model_feat_new_new_new_new.fit(xx)
            all_predictions = model_feat_new_new_new_new.predict(xx)
            x_train['feat_new_new_new_new']=all_predictions
            xx=pd.DataFrame(x_test[i])
            xx['zip']= x_test[j]
            all_predictions = model_feat_new_new_new_new.predict(xx)
            x_test['feat_new_new_new_new']=all_predictions
        
        
            del(x_train['price'])
            parameters = {
                'alpha': np.logspace(-6,-3,70)#-4,-3,70
            }

            standard_pipeline12=Pipeline([
                ('poly_feat', PolynomialFeatures(2)),
                ('std_scaler', StandardScaler()),
                ('cv', LinearRegression()),    
            ])


#cv=GridSearchCV(Ridge(), parameters,cv=5)
            ll=standard_pipeline12.fit(x_train,np.log(y_train))

            test_r=ll.predict(x_test)
            name.append(str(i)+' ' + str(j)+ ' ' + str(k))
            s.append(mape(y_test, np.exp(test_r)))



bedrooms
bathrooms
sqft_lot
floors
waterfront
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated
zipcode
lat
long
ll
new
zl
feat
feat_new
feat_new_new
feat_new_new_new
feat_new_new_new_new


In [2847]:
#Это набор фичей, который я добавил с помощью моей функции выше
x_train['price']= y_train
xx=pd.DataFrame(x_train['lat'])
xx['zip']= x_train['ll']
#model_feat = KMeans(n_clusters=5) 
#model_feat.fit(xx)
all_predictions = model_feat.predict(xx)
x_train['feat']=all_predictions
xx=pd.DataFrame(x_test['lat'])
xx['zip']= x_test['ll']
all_predictions = model_feat.predict(xx)
x_test['feat']=all_predictions



In [2848]:
x_train['price']= y_train
xx=pd.DataFrame(x_train['zipcode'])
xx['zip']= x_train['new']
model_feat_new = KMeans(n_clusters=6) 
model_feat_new.fit(xx)
all_predictions = model_feat_new.predict(xx)
x_train['feat_new']=all_predictions
xx=pd.DataFrame(x_test['zipcode'])
xx['zip']= x_test['new']
all_predictions = model_feat_new.predict(xx)
x_test['feat_new']=all_predictions

In [2849]:
x_train['price']= y_train
xx=pd.DataFrame(x_train['zipcode'])
xx['zip']= x_train['bathrooms']
#model_feat_new_new = KMeans(n_clusters=4) 
#model_feat_new_new.fit(xx)
all_predictions = model_feat_new_new.predict(xx)
x_train['feat_new_new']=all_predictions
xx=pd.DataFrame(x_test['zipcode'])
xx['zip']= x_test['bathrooms']
all_predictions = model_feat_new_new.predict(xx)
x_test['feat_new_new']=all_predictions

In [2850]:
x_train['price']= y_train
xx=pd.DataFrame(x_train['long'])
xx['zip']= x_train['waterfront']
#model_feat_new_new_new = KMeans(n_clusters=3) 
#model_feat_new_new_new.fit(xx)
all_predictions = model_feat_new_new_new.predict(xx)
x_train['feat_new_new_new']=all_predictions
xx=pd.DataFrame(x_test['long'])
xx['zip']= x_test['waterfront']
all_predictions = model_feat_new_new_new.predict(xx)
x_test['feat_new_new_new']=all_predictions

In [2851]:
x_train['price']= y_train
xx=pd.DataFrame(x_train['ll'])
xx['zip']= x_train['lat']
model_feat_new_new_new_new = KMeans(n_clusters=4) 
model_feat_new_new_new_new.fit(xx)
all_predictions = model_feat_new_new_new.predict(xx)
x_train['feat_new_new_new_new']=all_predictions
xx=pd.DataFrame(x_test['ll'])
xx['zip']= x_test['lat']
all_predictions = model_feat_new_new_new.predict(xx)
x_test['feat_new_new_new_new']=all_predictions

In [2856]:
#Итоговое значение, LinearRegression работает лучше всего
#del(x_train['price'])
standard_pipeline12=Pipeline([
    ('poly_feat', PolynomialFeatures(2)),
    ('std_scaler', StandardScaler()),
    ('cv', LinearRegression()),    
])
ll=standard_pipeline12.fit(x_train,np.log(y_train))
test_r=ll.predict(x_test)
mape(y_test, np.exp(test_r))

12.774066329920213